In [18]:
import tensorflow as tf
from transformers import TFGPT2Model, GPT2Tokenizer
# TODO: SESUAIKAN DENGAN FILE H5
model_file_path = "C:/Users/Michael/Documents/Bangkit/Project Capstone/latest_gpt2_saved_model.h5"
#Use Custom Objects if there is transfer learning
model = tf.keras.models.load_model(model_file_path,
                                      custom_objects={
                                          "TFGPT2Model":TFGPT2Model
                                      })


In [19]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Set the tokenizer padder
tokenizer.add_special_tokens({"pad_token": "<pad>"})
tokenizer.padding_side = "left"

In [20]:
import re
#breakdown articles/posts into sentence
def post_to_sentences(posts):
    sentence_list = []
    for post in posts:
        post = str(post)  # Convert paragraph to string
        # Split text on whitespace after period or question mark,
        # unless preceeded by single letter or followed by word character
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', post)
        for sentence in sentences:
            sentence_list.append(sentence) 
    return sentence_list

In [28]:
import numpy as np
#test inference
def predict(example):
    #example = "action, adventure, rpg, strategy, simulation, puzzle, sports, shooter, fantasy, platformer, multiplayer, mmo, stealth, horror, indie, arcade, co-op, competitive, exploration, racing, fighting, sandbox, open world, roguelike, turn-based, real-time, sci-fi, futuristic, mystery, retro, war, historical, mmorpg, first-person, third-person, virtual reality, casual, esports, management, survival, dungeon crawler, steampunk, moba, artificial intelligence, space, medieval, post-apocalyptic, western, zombies, surreal, cyberpunk, educational, team-based, realistic, mythology, crime, world-building, anime, military, comedy, romance, sci-fi, fps, maze, superhero, psychological, construction, board game, card game, visual novel, text-based, social, grand strategy, strategic, beat 'em up, retrogaming, match-three, tycoon, tactical, rogue-lite, point-and-click, tower defense, clicker, endless runner, horde mode, rhythm, flight simulator, sports management, survival horror, educational game, trivia, stealth action, city builder, word game, space exploration, management simulation, massively multiplayer, comic book, real-time strategy, monster, fast-paced, space opera, farming, social deduction, robotics, pixel art, turn-based strategy, lgbtq+, metroidvania, sandbox survival, post-apocalyptic world, martial arts, mythological, space combat, medieval fantasy, superheroine, real-time tactics, psychological horror, alternate history, deck-building, competitive multiplayer, archaeology, alien invasion, pirates, gunfight, mech, post-nuclear, real-time strategy game, rhythm game, survival crafting, monster taming, social simulation, virtual pet game, mythology-based, open-world sandbox, horror survival, crime thriller, 3d platformer, space exploration game, military strategy, cooking simulation, educational puzzle, anime-style, real-time simulation, stealth game, board game adaptation, card battle, visual novel game, tactical rpg, robotic warfare, pixel graphics, turn-based tactics, match-3 puzzle, horror adventure, action rpg, point-and-click adventure, defense game, clicker game, endless running, base building, horde survival, rhythm action, flight simulation, sports simulation, survival horror game, educational video game, word puzzle, space trading, monster collecting, fast-paced action, space opera game, farming simulation, social deduction game, robotic puzzle, 2d platformer, lgbtq+ representation, metroidvania game, sandbox building, post-apocalyptic survival, martial arts combat, mythological creatures, space combat game, medieval fantasy world, superheroine protagonist, real-time tactics game, psychological horror game, alternate history setting, deck-building game, competitive multiplayer game, archaeological exploration, alien invasion game, pirate adventure, gunfight game, mech combat, post-nuclear world, real-time strategy gameplay, rhythm gameplay, survival crafting game, monster taming game, social simulation game, virtual pet simulator, mythology-based game, open-world sandbox game, horror survival game, crime thriller game, 3d platforming, space exploration gameplay, military strategy game, cooking simulation game, educational puzzle game, anime-style visuals, real-time simulation game, stealth gameplay, board game adaptation game, card battle game, visual novel gameplay, tactical rpg game, robotic warfare game, pixel art graphics, turn-based tactics"
    #"In the pursuit of personal growth and self-improvement, books can serve as invaluable guides, offering insights, inspiration, and practical wisdom."
    #"The true magic happens during the editing process, where skilled hands transform raw footage and images into works of art."
    # Preprocess data, predict, then interpret
    example = post_to_sentences([example])
    #print(example)
    tokenized_example = tokenizer(example, padding=True, truncation=True, max_length=100, return_tensors="tf")
    #print(tokenized_example)
    #make prediction
    example_ids = tokenized_example.input_ids
    example_attention_mask = tokenized_example.attention_mask
    predicted_probs = model([example_ids,example_attention_mask]) # pass both inputs here
    predicted_probs = predicted_probs.numpy()
    #logits = predicted_label.logits
    #predicted_label = tf.argmax(logits,axis=-1)
    # Map the predicted label index to the actual label
    #BUTUH PERBAIKAN, INI RASANYA ADA YANG SALAH

    #Untuk memproses multi- sentence
    sum_probs = np.sum(predicted_probs, axis=0)
    #print("Predicted prob each sentence: ", predicted_probs)
    #print("Predicted prob: ", sum_probs)
    #urutannya bener books cinema cooking gaming sport? soalnya kalo diacak predicted probnya tetep sama
    label_mapping = {0: 'Gaming', 1: 'Sports', 2: 'Cinema', 3: 'Books', 4: 'Cooking'}
    index = np.argmax(sum_probs)
    predicted_label = label_mapping[index]

    #Print the predicted label
    print("Predicted label:", predicted_label)
    print("Predicted prob: ", predicted_probs)

In [30]:
import numpy as np
#test inference
example = "In the pursuit of personal growth and self-improvement, books can serve as invaluable guides, offering insights, inspiration, and practical wisdom."
#"In the pursuit of personal growth and self-improvement, books can serve as invaluable guides, offering insights, inspiration, and practical wisdom."
#"The true magic happens during the editing process, where skilled hands transform raw footage and images into works of art."
print("Text to be predicted: ", example)
predict(example)

Text to be predicted:  In the pursuit of personal growth and self-improvement, books can serve as invaluable guides, offering insights, inspiration, and practical wisdom.
Predicted label: Books
Predicted prob:  [[5.6195091e-03 2.1280670e-04 3.6419453e-03 9.9003017e-01 4.9544155e-04]]
